In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from datetime import date, datetime
import time
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm
import pickle
import spacy
import random
import ast
import seaborn as sns
import networkx as nx

In [386]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
tqdm.pandas()

In [4]:
pd.set_option('display.max_colwidth', -1)

In [5]:
# getting the burst timeline..
df_tweets = pd.read_csv('/home/auppal8/HexagonData_new.csv',lineterminator='\n', index_col=None)

In [405]:
df_timeline = pd.read_csv('/home/auppal8/userTimelineData.csv',lineterminator='\n',)

In [7]:
df_following = pd.read_excel('/home/auppal8/updatedfollowingData1117.xlsx', index_col=0)

In [8]:
# following data for user...
df_300 = pd.read_csv("~/Following300_1220.csv",index_col=0)

In [355]:
# getting the few labelled 
df_char = pd.read_csv("/home/auppal8/twitter_juul/input/user_characteristics.csv",index_col=0,keep_default_na=False)

In [9]:
#no of hashtags per tweet
# url and then no of url per tweet
# no of @user-mention per tweet
# percentage of retweet per user (user retweets / total no of tweets)

In [10]:
def hashtag_count(df):
    if df is not np.nan:
        hashtags = ast.literal_eval(df)
        if (hashtags is not None):
            return (len(hashtags))
        else:
            return(0)
    else:
        return(0)

In [406]:
# count no of hashtags
df_timeline['hashtags'] = df_timeline['hashtags'].progress_apply(hashtag_count)


100%|██████████| 1344182/1344182 [00:11<00:00, 121886.33it/s]

In [208]:
# find the occurence of urls
df_timeline['urls_id'] = df_timeline['tweetText'].str.findall(r'(https?://\S+)')

In [12]:
# count the no of urls per tweet
df_timeline['urls'] = df_timeline['tweetText'].str.count(r'(https?://\S+)')

In [13]:
# count no of user mentiosn per tweet
df_timeline['author_mentions'] = df_timeline['tweetText'].str.count(r'(\@\w+)')

In [26]:
# just to get avergae user mentions per user
df_user = df_timeline.groupby(df_timeline['userID']).agg({'author_mentions': 'median',
                                                          'urls' : 'median',
                                                          'retweetCount' : 'sum'})

In [33]:
total = df_user['retweetCount'].sum()
df_user['retweetCount'] = df_user['retweetCount'].apply(lambda x: (x/total) * 100)

In [41]:
df_timeline.columns

Index(['favourites_count', 'friendList', 'friendsCount', 'hashtags',
       'imageurl', 'lang', 'retweetCount', 'retweeted', 'tweetCreatedAt',
       'tweetId', 'tweetText', 'userCreatedAt', 'userDescription',
       'userFollowersCount', 'userID', 'userLocation', 'userName', 'urls',
       'author_mentions'],
      dtype='object')

In [40]:
# start with behavious characteristics
# max, min and average no of tweets per day  --done
# max, min time , average time interval between consecutive tweets --done
# total no of tweets
# no of unique urls

In [48]:
df_timeline['tweetCreatedAt'] = pd.to_datetime(df_timeline.tweetCreatedAt)

In [89]:
df_tweets = df_timeline.groupby([df_timeline['userID'],df_timeline['tweetCreatedAt'].dt.date])['tweetId'].agg('count')


In [92]:
df_tweets = pd.DataFrame(df_tweets)

In [133]:
# avg, min and max tweets
df_user['min_tweets'] = df_tweets.reset_index(level=0).groupby(['userID']).min()
df_user['max_tweets'] = df_tweets.reset_index(level=0).groupby(['userID']).max()
df_user['avg_tweets'] = df_tweets.reset_index(level=0).groupby(['userID']).mean()

In [155]:
df_tweets['diff'] = df_tweets.reset_index(level=0).index

In [159]:
df_tweets['diff'] = abs(df_tweets['diff'] - df_tweets['diff'].shift())

In [191]:
df_tweets['diff'] = df_tweets['diff'].apply(lambda x: x.days)

In [228]:
# min , max , avg no of days in consecutive intervals..and total no of tweets
df_user['min_interval'] = df_tweets.reset_index(level=0).groupby(['userID'])['diff'].min()
df_user['max_interval'] = df_tweets.reset_index(level=0).groupby(['userID'])['diff'].max()
df_user['avg_interval'] = df_tweets.reset_index(level=0).groupby(['userID'])['diff'].mean()
df_user['total_tweets'] = df_tweets.groupby(['userID'])['tweetId'].sum()

In [229]:
df_user['total_urls'] = df_timeline.groupby(['userID']).urls_id.sum().apply(lambda x: len(set(x)))

In [282]:
df_userNames = (df_timeline[['userID','userName','imageurl']].drop_duplicates()).reset_index(drop=True)

In [285]:
df_user =  df_userNames.join(df_user, on='userID')

In [313]:
df_char.columns

Index(['userID', 'userName', 'Promoter/not', 'author_mentions', 'urls',
       'retweetCount', 'min_tweets', 'max_tweets', 'avg_tweets',
       'min_interval', 'max_interval', 'avg_interval', 'total_urls',
       'total_tweets'],
      dtype='object')

In [358]:
Y = df_char['Promoter/not']

In [319]:
X = df_char[df_char.columns[~df_char.columns.isin(['userID','userName','Promoter/not'])]]

In [ ]:
# repalcing the NA values with NAN
Y = Y.replace('NA',np.nan)

##### selcting that are not NaN

In [377]:
X_sub = (X[~Y.isin([np.nan])])

In [378]:
y_sub = Y[~Y.isin([np.nan])]

In [384]:
# logistic regression
# labelling done....
model = LogisticRegression(C=1.0)
model = model.fit(X_sub,y_sub)

/home/auppal8/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [394]:
y_pred = (model.predict(X_sub))

In [395]:
score = accuracy_score(Y_pred, y_sub)

### accuracy with the 57 labelled nodes..


In [396]:
score

0.8596491228070176

In [397]:
# can now find propability of the remaining nodes....

In [402]:
model.predict_proba(X)[3]

array([0.86769694, 0.13230306])

In [403]:
# start with algorithm